# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_v2 import *
from ENV_DETAILS import *

2023-12-06 10:40:48.139109: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\MountainCar\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/MountainCar/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_ppo/' # Linux

ENV = "MountainCarContinuous-v0"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env


<TimeLimit<OrderEnforcing<Continuous_MountainCarEnv<MountainCarContinuous-v0>>>>

In [4]:
s = env.observation_space.sample()
s

array([ 0.41031712, -0.021853  ], dtype=float32)

In [5]:
env.observation_space, env.observation_space.shape

(Box([-1.2  -0.07], [0.6  0.07], (2,), float32), (2,))

In [6]:
s = env.reset()
s[0]

-0.5637484

In [7]:
s = env.reset()
env.action_space, env.action_space, env.step(s)

(Box(-1.0, 1.0, (1,), float32),
 Box(-1.0, 1.0, (1,), float32),
 (array([-0.49702165, -0.00095033], dtype=float32),
  -0.02460867434653844,
  False,
  {}))

### Training/Hyperparam run

In [8]:
#sudo rm /usr/lib/python3.11/EXTERNALLY-MANAGED

%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [9]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_PPO/"

In [10]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_ppo"
    project_name = "keras_tunning_ppo"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo/", evaluation_epoch = env._max_episode_steps, training_steps = 700000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.96, discount_max = 0.99,
                gae_min = 0.94, gae_max = 0.96,
                lr_actor_min = 0.0001, lr_actor_max = 0.001,
                lr_critic_min = 0.00001, lr_critic_max = 0.0001,
                entropy_factor_min = 0.01, entropy_factor_max = 0.1,
                dense_min = 32, dense_max = 256,
                environment_name=ENV,
                reward_scaler = 1, num_layers_act = 2, num_layers_crit =2, training_epoch = 50, memory_size= env._max_episode_steps),#env._max_episode_steps
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 20, 
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")
        writer= "logs_hyper/PPO/V0_"#Training/fit_ppo/"


        training_steps = 1000000
        entropy_factor = 0.01
        discount = 0.99
        dense_units_actor = [128]
        num_layers_actor = 1
        dense_units_critic = [128]
        num_layers_crit =1

        model = run_training(training_steps,  discount, dense_units_actor,  dense_units_critic, num_layers_actor, num_layers_crit, writer, 
                      environment_name = ENV,reward_scaler = 1, return_agent = True, lr_actor= 0.0001, lr_critic= 0.0001, sucess_criteria_value =SUCESS_CRITERIA_VALUE, 
                      gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 50)
 

Trial 20 Complete [00h 58m 43s]
total_train_reward: 26.36090569983596

Best total_train_reward So Far: 60.71634162347852
Total elapsed time: 1d 12h 01m 44s


In [11]:
exploration_tech = "soft"
hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=3):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

Trial id :03 | Score :60.71634162347852 --> {'discount': 0.99, 'gae_lambda': 0.95, 'lr_actor': 0.0008834281502792405, 'lr_critic': 2.2000165004917986e-05, 'policy_clip': 0.2, 'entropy_coeff': 0.04631515788482325, 'n_dense_layers_actor': 1, 'dense_units_act_0': 224, 'dense_units_act_1': 192, 'n_dense_layers_critic': 2, 'dense_units_crit_0': 256, 'dense_units_crit_1': 64}
Trial id :04 | Score :55.88987698197061 --> {'discount': 0.98, 'gae_lambda': 0.95, 'lr_actor': 0.0006504790532723968, 'lr_critic': 7.158270757627727e-05, 'policy_clip': 0.1, 'entropy_coeff': 0.02102833118576715, 'n_dense_layers_actor': 2, 'dense_units_act_0': 32, 'dense_units_act_1': 160, 'n_dense_layers_critic': 1, 'dense_units_crit_0': 32, 'dense_units_crit_1': 64}
Trial id :09 | Score :52.750280949799354 --> {'discount': 0.96, 'gae_lambda': 0.96, 'lr_actor': 0.0009642928731290008, 'lr_critic': 3.7032613506829866e-05, 'policy_clip': 0.2, 'entropy_coeff': 0.0772755472270666, 'n_dense_layers_actor': 1, 'dense_units_act_

In [12]:
val_env = gym.make(ENV)#, render_mode = "rgb_array"
dir = r"Hyperparam_kt_ppo"

for trials in tuner.oracle.get_best_trials(num_trials=1):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
    
env_model = tuner.get_best_models()[0]
final_rewards = final_evaluation(env_model,val_env,n_tries=1, exploration=exploration_tech,  video_name = "./ppo_v2_"+exploration_tech+"_video.mp4")

Trial id :03 | Score :60.71634162347852 --> {'discount': 0.99, 'gae_lambda': 0.95, 'lr_actor': 0.0008834281502792405, 'lr_critic': 2.2000165004917986e-05, 'policy_clip': 0.2, 'entropy_coeff': 0.04631515788482325, 'n_dense_layers_actor': 1, 'dense_units_act_0': 224, 'dense_units_act_1': 192, 'n_dense_layers_critic': 2, 'dense_units_crit_0': 256, 'dense_units_crit_1': 64}
Trial number :  20


In [13]:
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))


Final mean reward ' soft ': 76.61673795898703


In [14]:
final_rewards

76.61673795898703